In [1]:
import pandas as pd
import torch
import numpy as np
import utils
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from utils import *
seed_everything()

### Siema!
Warsztat KNUM 05.04.2019.

Do zrobienia:
1. Zaimplementuj early stopping. Trzymaj licznik, od ilu epok loss nie spada, jeśli przekroczy daną liczbę epok, to kończ trening.
2. Zmniejsz lub zwiększ learning rate. Czy loss nadal spada? Szybciej czy wolniej?
3. Przetestuj inne warotści batch size.
4. Wypróbuj głębsze modele (np 3, 4 albo 8 warstw). Zobacz, czy zachodzi overfitting.
5. Dodaj regularuzację L2. Dodaje się ją do optimizera za pomocą argumentu `weight_decay` w konstruktorze. Czy nadal zachodzi overfitting? Czy sieć uczy się tak samo szybko?
6. Powiększ jeszcze sieć. Jakie są wyniki? Do powiększonej sieci dodaj warstwę dropout, wypróbuj go z różnymi wartościami prawdopodobieńśtwa wyzerowania wagi.


In [2]:
class BaselineRegressor(nn.Module):
    '''
    Baseline Reggressor architecture
    input [13] -> [13] -> [1] output
    '''

    def __init__(self):
        super(BaselineRegressor, self).__init__()
        self.layer1 = nn.Linear(13, 13)
        self.layer2 = nn.Linear(13, 1)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = self.layer2(x)
        return x

In [3]:
def train(
    model,
    batch_size=32,
    learning_rate=3e-3,
    n_epochs=5,
):
    '''
    This function trains the model on boston housing dataset.
    At each epoch it trains model on train set and then evaluates if on test set.
    Function implements early stopping, when model overfits training shutdowns.

    :param model: Model instance to be trained
    :param batch_size: Size of batch
    :param learning_rate: Learning rate for optimizer (Adam)
    :param n_epochs: Number of training epochs
    :return: (best model, loss of best model)
    '''
    train_set, val_set = load_data()

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(val_set, batch_size=batch_size)

    optimizer = optim.Adam(model.parameters(),
                           lr=learning_rate,
                           )
    criterion = nn.MSELoss()

    for epoch in range(n_epochs):

        # train loop
        model.train()
        for x, y in train_loader:
            model.zero_grad()
            y = y.view(-1, 1)
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()

        # evaluation loop
        loss_sum = 0
        model.eval()
        with torch.no_grad():
            for x, y in test_loader:
                y = y.view(-1, 1)
                pred = model(x)
                loss = criterion(pred, y)
                loss_sum += loss.item()

        avg_loss = loss_sum / (len(val_set) / batch_size)
        print(f"Epoch: {epoch} loss: {avg_loss}")


    return model, avg_loss


In [4]:
seed_everything()
model = BaselineRegressor()
model, loss = train(
    model,
    batch_size=5,
    learning_rate=1e-3,
    n_epochs=50
)

print(model, loss)

Epoch: 0 loss: 73.00936924190972
Epoch: 1 loss: 48.95055222699023
Epoch: 2 loss: 46.227620229946346
Epoch: 3 loss: 41.933897743074915
Epoch: 4 loss: 47.38390167867105
Epoch: 5 loss: 41.55116002390704
Epoch: 6 loss: 42.207674548381895
Epoch: 7 loss: 40.1064343339815
Epoch: 8 loss: 51.82494223587157
Epoch: 9 loss: 45.75264461397186
Epoch: 10 loss: 45.000009649381866
Epoch: 11 loss: 40.2383227235689
Epoch: 12 loss: 37.05767787347629
Epoch: 13 loss: 38.56088262843335
Epoch: 14 loss: 38.417118853471415
Epoch: 15 loss: 40.52827395792083
Epoch: 16 loss: 35.22653767443079
Epoch: 17 loss: 31.567960761663485
Epoch: 18 loss: 32.40372684058242
Epoch: 19 loss: 40.41415800259808
Epoch: 20 loss: 37.23214757724071
Epoch: 21 loss: 35.210369215236874
Epoch: 22 loss: 31.010203061141368
Epoch: 23 loss: 27.352968764117385
Epoch: 24 loss: 26.818615451572448
Epoch: 25 loss: 25.84191115822379
Epoch: 26 loss: 35.94636909605011
Epoch: 27 loss: 31.879872487285947
Epoch: 28 loss: 25.71767112401527
Epoch: 29 loss: